In [1]:
cd /Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/AgNN

/Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/AgNN


In [2]:
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import GPyOpt

import os

import matplotlib as mpl
import matplotlib.tri as tri

import ternary
import pickle
import datetime

from collections import Counter
import matplotlib.ticker as ticker

import pyDOE
import random
from scipy.stats import norm

from sklearn.ensemble import RandomForestRegressor

In [3]:
dataset = pd.read_csv(r'/Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/AgNN/AgBO_loss.csv')
X = dataset.iloc[:,4:10]
X



,QAgNO3(%),Qpva(%),Qtsc(%),Qseed(%),Qtot(uL/min),loss
0,13.101322,32.995595,5.603524,4.528634,227.0,0.579592
1,13.101322,32.995595,5.603524,4.528634,227.0,0.581558
2,13.101322,32.995595,5.603524,4.528634,227.0,0.582454
3,13.101322,32.995595,5.603524,4.528634,227.0,0.582043
4,13.101322,32.995595,5.603524,4.528634,227.0,0.592302
...,...,...,...,...,...,...
3290,36.501053,14.000000,10.501053,4.501053,950.0,0.264635
3291,36.501053,14.000000,10.501053,4.501053,950.0,0.265406
3292,36.501053,14.000000,10.501053,4.501053,950.0,0.263623
3293,36.501053,14.000000,10.501053,4.501053,950.0,0.252167


In [4]:
X_run = X.groupby(['QAgNO3(%)', 'Qpva(%)', 'Qtsc(%)', 'Qseed(%)', 'Qtot(uL/min)'])['loss'].agg(lambda x: x.unique().mean())
X_run = (X_run.to_frame()).reset_index()
X_run

,QAgNO3(%),Qpva(%),Qtsc(%),Qseed(%),Qtot(uL/min),loss
0,4.530000,21.000000,6.670000,19.500000,600.0,0.875957
1,5.598775,19.001531,9.871363,5.598775,653.0,0.793268
2,5.869822,18.599606,4.228797,13.100592,507.0,0.901779
3,6.429792,34.800977,18.400488,17.599512,819.0,0.905287
4,6.672673,39.003003,16.300300,17.297297,333.0,0.854642
...,...,...,...,...,...,...
159,35.500000,25.000000,25.500000,0.500000,400.0,0.768887
160,36.500000,22.000000,2.500000,6.500000,900.0,0.169340
161,36.501053,14.000000,10.501053,4.501053,950.0,0.366709
162,38.421553,38.529319,4.301109,7.990491,631.0,0.232589


In [5]:
X_run['Qtot(uL/min)'] = X_run['Qtot(uL/min)'] / 30
X_run

,QAgNO3(%),Qpva(%),Qtsc(%),Qseed(%),Qtot(uL/min),loss
0,4.530000,21.000000,6.670000,19.500000,20.000000,0.875957
1,5.598775,19.001531,9.871363,5.598775,21.766667,0.793268
2,5.869822,18.599606,4.228797,13.100592,16.900000,0.901779
3,6.429792,34.800977,18.400488,17.599512,27.300000,0.905287
4,6.672673,39.003003,16.300300,17.297297,11.100000,0.854642
...,...,...,...,...,...,...
159,35.500000,25.000000,25.500000,0.500000,13.333333,0.768887
160,36.500000,22.000000,2.500000,6.500000,30.000000,0.169340
161,36.501053,14.000000,10.501053,4.501053,31.666667,0.366709
162,38.421553,38.529319,4.301109,7.990491,21.033333,0.232589


In [6]:
features = ['QAgNO3(%)', 'Qpva(%)', 'Qtsc(%)', 'Qseed(%)', 'Qtot(uL/min)']
X_feature = X_run[['QAgNO3(%)', 'Qpva(%)', 'Qtsc(%)', 'Qseed(%)', 'Qtot(uL/min)']].values

X_loss = np.array(X_run['loss'].values)

In [7]:
bounds = [{'name':'QAgNO3','type':'continuous','domain':(0.5,80)},
          {'name':'Qpva','type':'continuous','domain':(10,40)},
          {'name':'QNaOH','type':'continuous','domain':(0.5,80)},
          {'name':'Qhydra','type':'continuous','domain':(0.5,80)},
          {'name':'Qoil','type':'continuous','domain':(100,1000)}
         ]

constraints_baseline = [{'name': 'constr_1', 'constraint': 'x[:,0] + x[:,1] + x[:,2] + x[:,3] - 90'}
                       ]


In [8]:
# total 164 pts
n_initial = 14
n_test = int(np.round(len(X_loss) * 0.1))
jitter = 0.01

top10_indices = list(X_run.sort_values('loss').head(10).index)

seed_list = [74,660,861,1526,2,151,19,7,33,4021,66,42424,671,25,95711,850,71,2148,26219,4]
# 20

n_estimators = 50

In [15]:


def EI(X, model, y_best, input_mean, input_std):
#     xi = 0.01
    normalized_mean, normalized_std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
    
    mean = normalized_mean * input_std + input_mean
    std = normalized_std * input_std
    
#     z = (mean - y_best - xi)/std   "this is for maximization"
#     return (mean - y_best - xi) * norm.cdf(z) + std * norm.pdf(z)
#     z = (y_best - mean + xi)/std
#     return (y_best - mean + xi) * norm.cdf(z) + std * norm.pdf(z)
    z = (y_best - mean)/std
    return (y_best - mean) * norm.cdf(z) + std * norm.pdf(z)

def LCB(X, model, input_mean, input_std):
    normalized_mean, normalized_std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
    mean = normalized_mean * input_std + input_mean
    std = normalized_std * input_std
    
    return 2 * std - mean

def MPI(X, model, y_best, input_mean, input_std):
    normalized_mean, normalized_std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
    
    mean = normalized_mean * input_std + input_mean
    std = normalized_std * input_std
    
    z = (y_best - mean)/std
    return norm.cdf(z)

def calc_MAE(test_index, model, input_mean, input_std):
    total_mae = 0
    for index in test_index:
        X = X_feature[index]
        y = X_loss[index]
        normalized_mean, normalized_std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
        mean = normalized_mean * input_std + input_mean
        std = normalized_std * input_std
        mae_ = (y - mean)**2
        total_mae += mae_
    return total_mae / len(test_index)

def Top10Percent(top10_index, index_):
    how_many = 0
    for i in index_:
        if i in top10_index:
            how_many += 1
    return how_many
 


In [35]:
index_collection = []
X_collection = []
loss_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top10_index = top10_indices
    X_top10 = []
    loss_top10 = []
    for i in top10_index:
        X_top10.append(X_feature[i])
        loss_top10.append(X_loss[i])  
    
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    loss_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        loss_.append(X_loss[i])
        indices.remove(i)
        
    test_index = random.sample(indices, n_test)
    X_test = []
    loss_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        loss_test.append(X_loss[i])
        indices.remove(i)
        
#     print(index_)
    
    
        
#     print(test_index)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
#     print(indices)
    
    for i in np.arange(n_opt):
        
        input_mean = np.mean(loss_)
        input_std = np.std(loss_)
        input_best = np.min(loss_)
        
        BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                                   domain = bounds,
                                                   constraints = constraints_baseline,
                                                   acquisition_type = 'EI',              
                                                   normalize_Y = True,
                                                   X = np.array(X_),
                                                   Y = np.array([[i] for i in loss_]),
                                                   evaluator_type = 'local_penalization',
                                                   batch_size = 1,
                                                   acquisition_jitter = jitter)

        _ = BO_model.suggest_next_locations()
        
        next_index = None
        max_ac = -10**15
        for j in indices:
            X_j = X_feature[j]
            loss_j = X_loss[j]
            
            ac_value = EI(X_j, BO_model.model, input_best, input_mean, input_std)
#             print(ac_value)
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
#         print(next_index)        
        X_.append(X_feature[next_index])
        loss_.append(X_loss[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, BO_model.model, input_mean, input_std)
        MAE_.append(mae)
        how_many = Top10Percent(top10_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    loss_collection.append(loss_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, loss_collection, InTopPercent_collection, MAE_collection])
np.save('BO_EI_master', master)
   

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [32]:
index_collection = []
X_collection = []
loss_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top10_index = top10_indices
    X_top10 = []
    loss_top10 = []
    for i in top10_index:
        X_top10.append(X_feature[i])
        loss_top10.append(X_loss[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    loss_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        loss_.append(X_loss[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    loss_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        loss_test.append(X_loss[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_mean = np.mean(loss_)
        input_std = np.std(loss_)
        input_best = np.min(loss_)
        
        BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                                   domain = bounds,
                                                   constraints = constraints_baseline,
                                                   acquisition_type = 'LCB',              
                                                   normalize_Y = True,
                                                   X = np.array(X_),
                                                   Y = np.array([[i] for i in loss_]),
                                                   evaluator_type = 'local_penalization',
                                                   batch_size = 1,
                                                   acquisition_jitter = jitter)

        _ = BO_model.suggest_next_locations()
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            loss_j = X_loss[j]
            
            ac_value = LCB(X_j, BO_model.model, input_mean, input_std)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        loss_.append(X_loss[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, BO_model.model, input_mean, input_std)
        MAE_.append(mae)
        how_many = Top10Percent(top10_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    loss_collection.append(loss_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
  
    
master = np.array([index_collection, X_collection, loss_collection, InTopPercent_collection, MAE_collection])
np.save('BO_LCB_master', master)

initializing seed = 0
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The se

In [33]:
index_collection = []
X_collection = []
loss_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top10_index = top10_indices
    X_top10 = []
    loss_top10 = []
    for i in top10_index:
        X_top10.append(X_feature[i])
        loss_top10.append(X_loss[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    loss_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        loss_.append(X_loss[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    loss_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        loss_test.append(X_loss[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_mean = np.mean(loss_)
        input_std = np.std(loss_)
        input_best = np.min(loss_)
        
        BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                                   domain = bounds,
                                                   constraints = constraints_baseline,
                                                   acquisition_type = 'MPI',              
                                                   normalize_Y = True,
                                                   X = np.array(X_),
                                                   Y = np.array([[i] for i in loss_]),
                                                   evaluator_type = 'local_penalization',
                                                   batch_size = 1,
                                                   acquisition_jitter = jitter)

        _ = BO_model.suggest_next_locations()
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            loss_j = X_loss[j]
            
            ac_value = MPI(X_j, BO_model.model, input_best, input_mean, input_std)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        loss_.append(X_loss[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, BO_model.model, input_mean, input_std)
        MAE_.append(mae)
        how_many = Top10Percent(top10_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    loss_collection.append(loss_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, loss_collection, InTopPercent_collection, MAE_collection])
np.save('BO_MPI_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [34]:
# random sampling

index_collection = []
X_collection = []
loss_collection = []
InTopPercent_collection = []
MAE_collection = []

for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top10_index = top10_indices
    X_top10 = []
    loss_top10 = []
    for i in top10_index:
        X_top10.append(X_feature[i])
        loss_top10.append(X_loss[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    loss_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        loss_.append(X_loss[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    loss_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        loss_test.append(X_loss[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial

    for i in np.arange(n_opt):
    
        input_mean = np.mean(loss_)
        input_std = np.std(loss_)
        input_best = np.min(loss_)
        
        BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                                   domain = bounds,
                                                   constraints = constraints_baseline,
                                                   acquisition_type = 'EI',              
                                                   normalize_Y = True,
                                                   X = np.array(X_),
                                                   Y = np.array([[i] for i in loss_]),
                                                   evaluator_type = 'local_penalization',
                                                   batch_size = 1,
                                                   acquisition_jitter = jitter)

        _ = BO_model.suggest_next_locations()
        
        next_index = (random.sample(indices, 1))[0]
                
        X_.append(X_feature[next_index])
        loss_.append(X_loss[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, BO_model.model, input_mean, input_std)
        MAE_.append(mae)
        how_many = Top10Percent(top10_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    loss_collection.append(loss_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
master = np.array([index_collection, X_collection, loss_collection, InTopPercent_collection, MAE_collection])
np.save('RS_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [16]:
def EI(X, RF_model, y_best):

    
    tree_predictions = []
    for j in np.arange(n_estimators):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]

    
    std = np.std(np.array(tree_predictions), axis=0)[0]
    
    z = (y_best - mean)/std
    return (y_best - mean) * norm.cdf(z) + std * norm.pdf(z)

def LCB(X, RF_model):
    
    tree_predictions = []
    for j in np.arange(n_estimators):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]

    
    std = np.std(np.array(tree_predictions), axis=0)[0]

    
    return 2 * std - mean

def MPI(X, RF_model, y_best):
    
    tree_predictions = []
    for j in np.arange(n_estimators):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]
    
    std = np.std(np.array(tree_predictions), axis=0)[0]

    
    z = (y_best - mean)/std
    return norm.cdf(z)

def calc_MAE(test_index, RF_model):
    total_mae = 0
    for index in test_index:
        X = X_feature[index]
        y = X_loss[index]
        
        tree_predictions = []
        for j in np.arange(n_estimators):
            tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
        mean = np.mean(np.array(tree_predictions), axis=0)[0]

        std = np.std(np.array(tree_predictions), axis=0)[0]
        
        mae_ = (y - mean)**2
        total_mae += mae_
    return total_mae / len(test_index)

In [17]:
index_collection = []
X_collection = []
loss_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top10_index = top10_indices
    X_top10 = []
    loss_top10 = []
    for i in top10_index:
        X_top10.append(X_feature[i])
        loss_top10.append(X_loss[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    loss_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        loss_.append(X_loss[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    loss_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        loss_test.append(X_loss[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_best = np.min(loss_)
        
        RF_model = RandomForestRegressor(n_estimators=50)
        RF_model.fit(X_, loss_)
        
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            loss_j = X_loss[j]
            
            ac_value = EI(X_j, RF_model, input_best)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        loss_.append(X_loss[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, RF_model)
        MAE_.append(mae)
        how_many = Top10Percent(top10_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    loss_collection.append(loss_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, loss_collection, InTopPercent_collection, MAE_collection])
np.save('RF_EI_master', master)


initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [19]:
index_collection = []
X_collection = []
loss_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top10_index = top10_indices
    X_top10 = []
    loss_top10 = []
    for i in top10_index:
        X_top10.append(X_feature[i])
        loss_top10.append(X_loss[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    loss_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        loss_.append(X_loss[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    loss_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        loss_test.append(X_loss[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_best = np.min(loss_)
        
        RF_model = RandomForestRegressor(n_estimators=50)
        RF_model.fit(X_, loss_)
        
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            loss_j = X_loss[j]
            
            ac_value = LCB(X_j, RF_model)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        loss_.append(X_loss[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, RF_model)
        MAE_.append(mae)
        how_many = Top10Percent(top10_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    loss_collection.append(loss_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, loss_collection, InTopPercent_collection, MAE_collection])
np.save('RF_LCB_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [20]:
index_collection = []
X_collection = []
loss_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top10_index = top10_indices
    X_top10 = []
    loss_top10 = []
    for i in top10_index:
        X_top10.append(X_feature[i])
        loss_top10.append(X_loss[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    loss_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        loss_.append(X_loss[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    loss_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        loss_test.append(X_loss[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_best = np.min(loss_)
        
        RF_model = RandomForestRegressor(n_estimators=50)
        RF_model.fit(X_, loss_)
        
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            loss_j = X_loss[j]
            
            ac_value = MPI(X_j, RF_model, input_best)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        loss_.append(X_loss[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, RF_model)
        MAE_.append(mae)
        how_many = Top10Percent(top10_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    loss_collection.append(loss_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, loss_collection, InTopPercent_collection, MAE_collection])
np.save('RF_MPI_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
